<h1 align="center"> Aplicações em Processamento de Linguagem Natural </h1>
<h2 align="center"> Aula 07 - Extração de Informação (Parte 2)</h2>
<h3 align="center"> Prof. Fernando Vieira da Silva MSc.</h3>

<h2>1. Extração de Relacionamentos</h2>
<p>A extração de relacionamentos consiste em identificar a ligação entre diversas entidades nomeadas no texto. Isso envolve mencionar qual é o tipo da ligação entre duas entidades. Considere o exemplo de sentença abaixo.</p>

<p>"Carlos Alberto de Nogueira é o morador mais antigo da Rua Praça da Alegria."</p>

<p>Temos as entidades:</p>

* Carlos Alberto de Nogueira (PESSOA)
* Rua Praça da Alegria (LOCALIDADE)

<p>Essas mesmas entidades estão relacionadas da seguinte forma:</p>

[Carlos Alberto de Nogueira (PERSON); morador mais antigo; Rua Praça da Alegria (LOCALIDADE)]


<p>Um dos mais famosos exemplos de sistema de reconhecimento é o [Never-Ending Language Learning (NELL)](http://rtw.ml.cmu.edu/), projeto desenvolvido pela Universidade Carnigie Mellon, com participação do Google e inclusive de pesquisadores brasileiros financiados pelo CNPq. Esse projeto consiste em extrair relacionamentos de milhões de páginas da internet, criando uma gigantesca base de conhecimento.</p>

<h2>2. Métodos para identificação de relacionamentos</h2>

<p>Os métodos mais comuns para identificar relacionamentos entre entidades são:</p>

* **Padrões codificados manualmente**: Basta criar padrões usando expressões regulares, por exemplo, para identificar que duas entidades se relacionam. Assim como em "X mora em Y" pode ser um padrão para identificar o relacionamento (X, mora_em, Y) entre uma entidade X do tipo PESSOA e uma entidade Y do tipo LOCALIDADE.
* **Métodos bootstraping**: Com poucos dados, procura por ocorrências de duas entidades em que já se conhece o relacionamento (no Google, por exemplo), e usa os modelos encontrados como modelos para o mesmo relacionamento entre outras entidades.
* **Métodos supervisionados**: Com base num corpus anotado com relacionamentos, criar modelos que 1) detecte quando existe o relacionamento entre duas entidades e 2) classifique o tipo de relacionamento entre elas. 

<p>Nesta aula, vamos ver um método supervisionado para classificar o relacionamento entre entidades, usando técnicas que já utilizamos em aulas anteriores.</p>

<p>Para isso, utilizaremos alguns atributos mais comuns para o problema, como:</p>

* Bag of Words/LSA
* Flags indicadores dos tipos das entidades
* Número de palavras entre as duas entidades
* Flag indicando se o texto de uma entidade é composto pelo texto da outra
* POS tags
* etc



<h2>3. Criando um Modelo Supervisionado</h2>
<p> Vamos utilizar o corpus [Figure Eight: Medical Sentence Summary](https://www.kaggle.com/kmader/figure-eight-medical-sentence-summary), que possui diversas sentenças extraídas do PubMed, com entidades anotadas, assim como seus tipos de relacionamento.</p>

In [26]:
import pandas as pd

df_train = pd.read_csv('../input/figure-eight-medical-sentence-summary/train.csv')
df_test = pd.read_csv('../input/figure-eight-medical-sentence-summary/test.csv')

df_train.head(20)

,_unit_id,_created_at,_canary,_id,_started_at,_channel,_trust,_worker_id,_country,_region,_city,_ip,direction,b1,b2,direction_gold,e1,e2,relation,relex_relcos,sent_id,sentence,term1,term2,twrex
0,502808352,7/13/2014 13:48:35,NaN,1321892767,7/13/2014 13:48:14,clixsense,0.9167,27871219,NLD,07,Amsterdam,87.210.207.223,IM CEFTRIAXONE treats URETHRAL OR RECTAL GONOR...,41,128,NaN,69,142,treats,1.000000,907845-FS1-2,"For treatment of uncomplicated cervical, URETH...",URETHRAL OR RECTAL GONORRHEA,IM CEFTRIAXONE,RO-may_treat
1,502808352,7/13/2014 13:51:12,NaN,1321894040,7/13/2014 13:51:07,neodev,0.8333,17610000,GBR,I2,Manchester,90.200.140.201,URETHRAL OR RECTAL GONORRHEA treats IM CEFTRIA...,41,128,NaN,69,142,treats,1.000000,907845-FS1-2,"For treatment of uncomplicated cervical, URETH...",URETHRAL OR RECTAL GONORRHEA,IM CEFTRIAXONE,RO-may_treat
2,502808352,7/13/2014 16:24:57,NaN,1321961909,7/13/2014 16:24:35,instagc,0.6639,25990856,USA,NV,Las Vegas,68.108.98.78,IM CEFTRIAXONE treats URETHRAL OR RECTAL GONOR...,41,128,NaN,69,142,treats,1.000000,907845-FS1-2,"For treatment of uncomplicated cervical, URETH...",URETHRAL OR RECTAL GONORRHEA,IM CEFTRIAXONE,RO-may_treat
3,502808352,7/13/2014 16:33:49,NaN,1321965723,7/13/2014 16:33:31,elite,0.3923,28276268,USA,CA,San Diego,76.88.95.100,URETHRAL OR RECTAL GONORRHEA treats IM CEFTRIA...,41,128,NaN,69,142,treats,1.000000,907845-FS1-2,"For treatment of uncomplicated cervical, URETH...",URETHRAL OR RECTAL GONORRHEA,IM CEFTRIAXONE,RO-may_treat
4,502808352,7/13/2014 16:47:27,NaN,1321970904,7/13/2014 16:47:06,neodev,0.6552,27597779,CAN,AB,Calgary,68.146.86.137,IM CEFTRIAXONE treats URETHRAL OR RECTAL GONOR...,41,128,NaN,69,142,treats,1.000000,907845-FS1-2,"For treatment of uncomplicated cervical, URETH...",URETHRAL OR RECTAL GONORRHEA,IM CEFTRIAXONE,RO-may_treat
5,502808352,7/13/2014 16:56:13,NaN,1321973849,7/13/2014 16:55:37,clixsense,0.6639,28037714,GBR,I4,Mitcham,94.4.232.118,IM CEFTRIAXONE treats URETHRAL OR RECTAL GONOR...,41,128,NaN,69,142,treats,1.000000,907845-FS1-2,"For treatment of uncomplicated cervical, URETH...",URETHRAL OR RECTAL GONORRHEA,IM CEFTRIAXONE,RO-may_treat
6,502808352,7/13/2014 17:14:41,NaN,1321979856,7/13/2014 17:14:06,prodege,0.6151,2422962,USA,IA,Honey Creek,12.73.110.97,IM CEFTRIAXONE treats URETHRAL OR RECTAL GONOR...,41,128,NaN,69,142,treats,1.000000,907845-FS1-2,"For treatment of uncomplicated cervical, URETH...",URETHRAL OR RECTAL GONORRHEA,IM CEFTRIAXONE,RO-may_treat
7,502808354,7/13/2014 13:45:15,NaN,1321891302,7/13/2014 13:44:25,clixsense,0.9167,27871219,NLD,07,Amsterdam,87.210.207.223,no_relation,175,203,NaN,187,217,diagnosed by,0.530330,906321-FS1-13,Diagnosis specific malignancies available for ...,OSTEOSARCOMA,RETINOBLASTOMA,RO-has_manifestation
8,502808354,7/13/2014 13:50:45,NaN,1321893871,7/13/2014 13:50:40,neodev,0.8333,17610000,GBR,I2,Manchester,90.200.140.201,OSTEOSARCOMA diagnosed by RETINOBLASTOMA,175,203,NaN,187,217,diagnosed by,0.530330,906321-FS1-13,Diagnosis specific malignancies available for ...,OSTEOSARCOMA,RETINOBLASTOMA,RO-has_manifestation
9,502808354,7/13/2014 14:07:58,NaN,1321902037,7/13/2014 14:07:28,prodege,0.9444,23977248,GBR,B5,Wembley,82.28.55.95,no_relation,175,203,NaN,187,217,diagnosed by,0.530330,906321-FS1-13,Diagnosis specific malignancies available for ...,OSTEOSARCOMA,RETINOBLASTOMA,RO-has_manifestation


In [27]:
df_train['relation'].unique()

array(['treats', 'diagnosed by', 'contraindicates', 'causes', 'location',
       'is location of', 'location of', 'is diagnosed by',
       'diagnose_by_test_or_drug'], dtype=object)

<p>Transformamos as sentenças e tipos de relacionamento em matrizes numpy. Também binarizamos os rótulos dos relacionamentos, para utilizarmos no nosso classificador logo mais.</p>

In [28]:
import numpy as np

x_train = df_train['sentence'].as_matrix()
y_train = df_train['relation'].as_matrix()

from sklearn.preprocessing import label_binarize

y_train = label_binarize(y_train, classes=df_train['relation'].unique())

print(x_train[:10])
print(y_train[:10])

['For treatment of uncomplicated cervical, URETHRAL OR RECTAL GONORRHEA CDC and others recommend IM ceftriaxone or oral cefixime; IM CEFTRIAXONE is drug of choice for pharyngeal infections.'
 'For treatment of uncomplicated cervical, URETHRAL OR RECTAL GONORRHEA CDC and others recommend IM ceftriaxone or oral cefixime; IM CEFTRIAXONE is drug of choice for pharyngeal infections.'
 'For treatment of uncomplicated cervical, URETHRAL OR RECTAL GONORRHEA CDC and others recommend IM ceftriaxone or oral cefixime; IM CEFTRIAXONE is drug of choice for pharyngeal infections.'
 'For treatment of uncomplicated cervical, URETHRAL OR RECTAL GONORRHEA CDC and others recommend IM ceftriaxone or oral cefixime; IM CEFTRIAXONE is drug of choice for pharyngeal infections.'
 'For treatment of uncomplicated cervical, URETHRAL OR RECTAL GONORRHEA CDC and others recommend IM ceftriaxone or oral cefixime; IM CEFTRIAXONE is drug of choice for pharyngeal infections.'
 'For treatment of uncomplicated cervical, UR

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


<p>Como não temos os tipos das entidades, mas sabemos que se trata de nomes de medicamentos e doenças na maioria dos casos, não utilizaremos o tipo das entidades como atributos, mas utilizaremos os POS tags de todas as palavras entre as entidades. Vamos criar outras matrizes com esses atributos. </p>

<p>Para os POS Tags, vamos fazer algo parecido ao chunking sugerido em https://courses.cs.washington.edu/courses/cse517/13wi/slides/cse517wi13-RelationExtraction.pdf, mas ao invés de usar chunking, vamos criar 3-grams desses POS tags para simplificar.</p>

In [29]:
x_train_sub_list = []

for i, row in df_train.iterrows():
    pos_t1 = row['sentence'].find(row['term1'])
    pos_t2 = row['sentence'].find(row['term2'])    
    
    if pos_t1 < pos_t2:
        len_t1 = len(row['term1'])
        x_train_sub_list.append(row['sentence'][pos_t1+len_t1:pos_t2])
    else:
        len_t2 = len(row['term2'])
        x_train_sub_list.append(row['sentence'][pos_t2+len_t2:pos_t1])
        
x_train_sub = np.array(x_train_sub_list)

print(x_train_sub[:10])

[' CDC and others recommend IM ceftriaxone or oral cefixime; '
 ' CDC and others recommend IM ceftriaxone or oral cefixime; '
 ' CDC and others recommend IM ceftriaxone or oral cefixime; '
 ' CDC and others recommend IM ceftriaxone or oral cefixime; '
 ' CDC and others recommend IM ceftriaxone or oral cefixime; '
 ' CDC and others recommend IM ceftriaxone or oral cefixime; '
 ' CDC and others recommend IM ceftriaxone or oral cefixime; '
 " Wilms' tumour, " " Wilms' tumour, " " Wilms' tumour, "]


<p>Agora vamos definir duas funções de tokenização: uma para tokenizar bag-of-words e outra para tokenizar os POS tags</p>

In [30]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

def my_tokenizer_pos(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    return [pos[1] for pos in pos_tags]

# testando nossa função:

for x in x_train_sub[:10]:
    print(my_tokenizer_pos(x))

['NNP', 'CC', 'NNS', 'VBP', 'NNP', 'NN', 'CC', 'JJ', 'NN', ':']
['NNP', 'CC', 'NNS', 'VBP', 'NNP', 'NN', 'CC', 'JJ', 'NN', ':']
['NNP', 'CC', 'NNS', 'VBP', 'NNP', 'NN', 'CC', 'JJ', 'NN', ':']
['NNP', 'CC', 'NNS', 'VBP', 'NNP', 'NN', 'CC', 'JJ', 'NN', ':']
['NNP', 'CC', 'NNS', 'VBP', 'NNP', 'NN', 'CC', 'JJ', 'NN', ':']
['NNP', 'CC', 'NNS', 'VBP', 'NNP', 'NN', 'CC', 'JJ', 'NN', ':']
['NNP', 'CC', 'NNS', 'VBP', 'NNP', 'NN', 'CC', 'JJ', 'NN', ':']
['NNP', 'POS', 'NN', ',']
['NNP', 'POS', 'NN', ',']
['NNP', 'POS', 'NN', ',']


In [31]:
x_train_sub

array([' CDC and others recommend IM ceftriaxone or oral cefixime; ',
       ' CDC and others recommend IM ceftriaxone or oral cefixime; ',
       ' CDC and others recommend IM ceftriaxone or oral cefixime; ', ...,
       ' requirement in patients with ', ' requirement in patients with ',
       ' requirement in patients with '], dtype='<U261')

In [32]:
stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer_bow(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

<p>Vamos reaproveitar a classe para seleção de atributos usando SVD.</p>

In [33]:
from sklearn.decomposition import TruncatedSVD

class SVDDimSelect(object):
    def fit(self, X, y=None):        
        try:
            self.svd_transformer = TruncatedSVD(n_components=round(X.shape[1]/2))
            self.svd_transformer.fit(X)
        
            cummulative_variance = 0.0
            k = 0
            for var in sorted(self.svd_transformer.explained_variance_ratio_)[::-1]:
                cummulative_variance += var
                if cummulative_variance >= 0.5:
                    break
                else:
                    k += 1
                
            self.svd_transformer = TruncatedSVD(n_components=k)
        except Exception as ex:
            print(ex)
            
        return self.svd_transformer.fit(X)
    
    def transform(self, X, Y=None):
        return self.svd_transformer.transform(X)
        
    def get_params(self, deep=True):
        return {}

<p>Agora vamos criar nosso Pipeline. Em resumo, vamos usar o TFIDF Vectorizer e o nosso POS Tagger em paralelo, e depois juntar os atributos para redimensionar usando o SVD.</p>

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import scipy

clf = OneVsRestClassifier(LogisticRegression(random_state=0, solver='lbfgs', 
                                             multi_class='multinomial'))


my_pipeline = Pipeline([
                        ('union', FeatureUnion([('bow', TfidfVectorizer(tokenizer=my_tokenizer_bow)),\
                                                ('pos', Pipeline([('pos-vect', CountVectorizer(tokenizer=my_tokenizer_pos)), \
                                                         ('pos-tfidf', TfidfTransformer())]))
                                               ])),\
                       ('svd', SVDDimSelect()), \
                       ('clf', clf)])

par = {'clf__estimator__C' : np.logspace(-4, 4, 20)}

hyperpar_selector = RandomizedSearchCV(my_pipeline, par, cv=3, scoring='f1_weighted', n_jobs=1, n_iter=20)

<p>Agora vamos treinar os algoritmos</p>

In [35]:
print(x_train_sub.shape)
print(y_train.shape)

hyperpar_selector.fit(X=x_train_sub[:500], y=y_train[:500])

(13340,)
(13340, 9)


/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

...

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 7 is present in all train

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('bow', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_r...tate=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None))]),
          fit_params=None, iid='warn', n_iter=20, n_jobs=1,
          param_distributions={'clf__estimator__C': array([1.00000e-04, 2.63665e-04, 6.95193e-04, 1.83298e-03, 4.83293e-03,
       1.27427e-02, 3.35982e-02, 8.85867e-02, 2.33572e-01, 6.15848e-01,
       1.62378e+00, 4.28133e+00, 1.12884e+01, 2.97635e+01, 7.84760e+01,
       2.06914e+02, 5.45559e+02, 1.43845e+03, 3.79269e+03, 1.00000e+04])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_tra

In [36]:
x_test = df_test['sentence'].as_matrix()
y_test = df_test['relation'].as_matrix()

y_test = label_binarize(y_test, classes=df_train['relation'].unique())

x_test_sub_list = []

for i, row in df_test.iterrows():
    pos_t1 = row['sentence'].find(row['term1'])
    len_t1 = len(row['term1'])
    
    pos_t2 = row['sentence'].find(row['term2'])    
    
    x_test_sub_list.append(row['sentence'][pos_t1+len_t1:pos_t2])
    

x_test_sub = np.array(x_test_sub_list)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [37]:
y_predicted = hyperpar_selector.predict(x_test)

In [38]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predicted, target_names=df_train['relation'].unique()))

                          precision    recall  f1-score   support

                  treats       0.41      0.14      0.20      1680
            diagnosed by       0.11      0.03      0.05       224
         contraindicates       0.00      0.00      0.00        33
                  causes       0.57      0.22      0.32      2338
                location       0.00      0.00      0.00         7
          is location of       0.00      0.00      0.00       220
             location of       0.00      0.00      0.00         0
         is diagnosed by       0.00      0.00      0.00        64
diagnose_by_test_or_drug       0.00      0.00      0.00         0

               micro avg       0.49      0.17      0.25      4566
               macro avg       0.12      0.04      0.06      4566
            weighted avg       0.45      0.17      0.24      4566
             samples avg       0.17      0.17      0.17      4566



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


<p>
</p>

<p><b>Exercício 7:</b> Treine um modelo de extração de relacionamentos em Português, utilizando o corpus extraído do DBPedia e com relacionamentos entre pares de entidades anotadas.</p>

In [39]:
import pandas as pd
df_dict = {"SENTENCE":[],"ENTITY1":[],"ENTITY2":[],"REL TYPE":[]}
filepath = '../input/dbpedia-with-entity-relations-in-portuguese/DBpediaRelations-PT-0.2.txt'  
with open(filepath) as fp:  
   for cnt, line in enumerate(fp):
        for key, value in df_dict.items() :
            if (key in line.split(':')[0]):
                df_dict[key].append((' '.join(line.split(':')[1:]))[:-1])
df=pd.DataFrame.from_dict(df_dict) 
df.columns = ['sentence', 'term1', 'term2', 'relation']

In [40]:
x_sub_list = []
for i, row in df.iterrows():
    pos_t1 = row['sentence'].find(row['term1'])
    pos_t2 = row['sentence'].find(row['term2'])    
    if pos_t1 < pos_t2:
        len_t1 = len(row['term1'])
        x_sub_list.append(row['sentence'][pos_t1+len_t1:pos_t2])
    else:
        len_t2 = len(row['term2'])
        x_sub_list.append(row['sentence'][pos_t2+len_t2:pos_t1])
df['sen_sub'] = pd.Series(x_sub_list)
df.describe()

,sentence,term1,term2,relation,sen_sub
count,98023,98023,98023,98023,98023
unique,84382,64289,14074,10,39853
top,"Faz fronteira com Altidona, Belmonte Piceno, ...",Mossoró,Brasil,locatedInArea,
freq,16,37,2796,46864,39050


In [41]:
#verificar valores distintos de relação
df['relation'].value_counts()/df['relation'].shape[0]

 locatedInArea         0.478092
 origin                0.267651
 other                 0.108709
 deathOrBurialPlace    0.072901
 partOf                0.056313
 successor             0.005784
 keyPerson             0.003999
 parent                0.003040
 partner               0.001938
 influencedBy          0.001571
Name: relation, dtype: float64

In [42]:
# separar 70/30 de maneira estratificada e randomica
X = df['sen_sub'].values
y = df['relation'].values
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
y = lb.fit_transform(y)

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit (10, test_size=0.2, random_state=0)
a = sss.split(X, y)
for train_index, test_index in a:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


In [43]:
#VERIFICA SE AS CLASSES FORAM SEPARADAS DE MANEIRA PROPORCIONAL(estratificados) PARA TESTE
y_train_original = pd.DataFrame(lb.inverse_transform(y_train))
y_test_original = pd.DataFrame(lb.inverse_transform(y_test))
display(y_train_original[0].value_counts()/y_train_original[0].shape[0])
display(y_test_original[0].value_counts()/y_test_original[0].shape[0])

 locatedInArea         0.478092
 origin                0.267655
 other                 0.108712
 deathOrBurialPlace    0.072904
 partOf                0.056314
 successor             0.005777
 keyPerson             0.004004
 parent                0.003035
 partner               0.001938
 influencedBy          0.001569
Name: 0, dtype: float64

 locatedInArea         0.478092
 origin                0.267636
 other                 0.108697
 deathOrBurialPlace    0.072890
 partOf                0.056312
 successor             0.005815
 keyPerson             0.003979
 parent                0.003060
 partner               0.001938
 influencedBy          0.001581
Name: 0, dtype: float64

In [44]:
!python -m spacy download pt_core_news_sm

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [45]:
import nltk
from nltk import pos_tag 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

import pt_core_news_sm
nlp = pt_core_news_sm.load()

from nltk.stem import RSLPStemmer
stopwords_list = stopwords.words('portuguese')
stemmer = RSLPStemmer()

def my_tokenizer_bow_pt(doc):
    words = word_tokenize(doc, language ='portuguese')
    non_stopwords = [w for w in words if not w[0].lower() in stopwords_list]
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    stems = []
    for w in non_punctuation:
        stems.append(stemmer.stem(w))
    return stems

#testando o tokenizador adaptado para o portugues
print(my_tokenizer_bow_pt('O gato roeu a roupa do rei de roma'))

['gat', 'roeu', 'roup', 'do', 'rei', 'de', 'rom']


In [46]:
def my_tokenizer_pos_pt(doc):
    tokens = nlp(doc)
    pos_tags = []
    for t in tokens:
        pos_tags.append(t.pos_)
    return pos_tags

# testando nossa função:

for x in X_train[:10]:
    print(my_tokenizer_pos_pt(x))

[]
[]
['SPACE', 'PRON', 'VERB', 'ADP', 'NOUN', 'DET', 'NOUN', 'ADP', 'NOUN', 'X', 'ADP']
['SPACE', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'VERB', 'PRON', 'NOUN', 'ADP', 'NUM', 'DET', 'NOUN', 'PUNCT', 'NOUN', 'PROPN', 'PROPN', 'CCONJ', 'PROPN', 'NOUN', 'CCONJ', 'ADJ']
[]
['SPACE', 'VERB', 'PROPN', 'PROPN', 'ADJ', 'ADP', 'PROPN', 'PROPN', 'CCONJ', 'PROPN', 'PROPN', 'PUNCT', 'ADV', 'ADV', 'PROPN']
['SPACE', 'AUX', 'VERB', 'ADV', 'ADP', 'NUM', 'DET', 'NOUN', 'ADP']
['SPACE', 'DET', 'NOUN', 'AUX', 'VERB', 'NOUN', 'PROPN', 'PROPN', 'NUM', 'PUNCT', 'ADP', 'NUM', 'ADP', 'NOUN', 'ADP', 'NUM', 'PUNCT', 'ADP', 'NOUN', 'ADP', 'DET', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'ADP', 'NOUN', 'VERB', 'ADP', 'PROPN', 'ADP']
[]
['SPACE', 'VERB', 'ADV', 'ADP', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'NOUN', 'ADV', 'ADP', 'NUM', 'SYM', 'VERB', 'ADP', 'NOUN', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'VERB', 'ADP', 'DET', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'PRON']


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import scipy

clf = OneVsRestClassifier(LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial'))

my_pipeline = Pipeline([('union', FeatureUnion([('bow', TfidfVectorizer(tokenizer=my_tokenizer_bow_pt)),\
                                                ('pos', Pipeline([('pos-vect', CountVectorizer(tokenizer=my_tokenizer_pos_pt)), \
                                                         ('pos-tfidf', TfidfTransformer())]))
                                               ])),\
                       ('svd', SVDDimSelect()), \
                       ('clf', clf)])

par = {'clf__estimator__C' : np.logspace(-4, 4, 20)}

hyperpar_selector = RandomizedSearchCV(my_pipeline, par, cv=3, scoring='f1_weighted', n_jobs=1, n_iter=20)

In [48]:
hyperpar_selector.fit(X=X_train[:500], y=y_train[:500])

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

...

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  str(classes[c]))


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('bow', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_r...tate=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None))]),
          fit_params=None, iid='warn', n_iter=20, n_jobs=1,
          param_distributions={'clf__estimator__C': array([1.00000e-04, 2.63665e-04, 6.95193e-04, 1.83298e-03, 4.83293e-03,
       1.27427e-02, 3.35982e-02, 8.85867e-02, 2.33572e-01, 6.15848e-01,
       1.62378e+00, 4.28133e+00, 1.12884e+01, 2.97635e+01, 7.84760e+01,
       2.06914e+02, 5.45559e+02, 1.43845e+03, 3.79269e+03, 1.00000e+04])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_tra

In [49]:
y_predicted = hyperpar_selector.predict(X_test)

In [50]:
from sklearn.metrics import classification_report
print(classification_report(lb.inverse_transform(y_test), lb.inverse_transform(y_predicted)))

                     precision    recall  f1-score   support

 deathOrBurialPlace       0.16      0.57      0.25      1429
       influencedBy       0.00      0.00      0.00        31
          keyPerson       0.00      0.00      0.00        78
      locatedInArea       0.62      0.87      0.72      9373
             origin       0.53      0.10      0.16      5247
              other       0.00      0.00      0.00      2131
             parent       0.00      0.00      0.00        60
             partOf       0.28      0.07      0.11      1104
            partner       0.00      0.00      0.00        38
          successor       0.00      0.00      0.00       114

          micro avg       0.49      0.49      0.49     19605
          macro avg       0.16      0.16      0.12     19605
       weighted avg       0.46      0.49      0.41     19605



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
